In [ ]:
import numpy as np
import sympy as sp
from sympy import symbols
from sympy import Matrix
from sympy.printing import latex
import latexify


In [17]:
a, m, h, theta = symbols('a m h theta')

Ixx, Ixy, Ixz, Iyx, Iyy, Iyz, Izx, Izy, Izz = symbols('Ixx Ixy Ixz Iyx Iyy Iyz Izx Izy Izz')

# Inertia components for a single prism
I_xx = (5/24) * m * a**2
I_yy = (m * h**2) / 12 + (5/48) * m * a**2

# Total inertia tensor for the two prisms (centered at the same point)
I = np.array([
    [Ixx,       Ixy,          Ixz],
    [Ixy,        Iyy,         Iyz],
    [Ixz,            Iyz,      Izz]
])

#Rotation theta about x
r = np.array([
    [1,       0,          0],
    [0,        sp.cos(theta),         -sp.sin(theta)],
    [0,            sp.sin(theta),      sp.cos(theta)]
])

I_R = r @ I @ r.T

print(I + I_R)

[[2*Ixx Ixy*cos(theta) + Ixy - Ixz*sin(theta)
  Ixy*sin(theta) + Ixz*cos(theta) + Ixz]
 [Ixy*cos(theta) + Ixy - Ixz*sin(theta)
  Iyy + (Iyy*cos(theta) - Iyz*sin(theta))*cos(theta) - (Iyz*cos(theta) - Izz*sin(theta))*sin(theta)
  Iyz + (Iyy*cos(theta) - Iyz*sin(theta))*sin(theta) + (Iyz*cos(theta) - Izz*sin(theta))*cos(theta)]
 [Ixy*sin(theta) + Ixz*cos(theta) + Ixz
  Iyz + (Iyy*sin(theta) + Iyz*cos(theta))*cos(theta) - (Iyz*sin(theta) + Izz*cos(theta))*sin(theta)
  Izz + (Iyy*sin(theta) + Iyz*cos(theta))*sin(theta) + (Iyz*sin(theta) + Izz*cos(theta))*cos(theta)]]


In [18]:
Matrix(I_R)

Matrix([
[                            Ixx,                                                             Ixy*cos(theta) - Ixz*sin(theta),                                                             Ixy*sin(theta) + Ixz*cos(theta)],
[Ixy*cos(theta) - Ixz*sin(theta), (Iyy*cos(theta) - Iyz*sin(theta))*cos(theta) - (Iyz*cos(theta) - Izz*sin(theta))*sin(theta), (Iyy*cos(theta) - Iyz*sin(theta))*sin(theta) + (Iyz*cos(theta) - Izz*sin(theta))*cos(theta)],
[Ixy*sin(theta) + Ixz*cos(theta), (Iyy*sin(theta) + Iyz*cos(theta))*cos(theta) - (Iyz*sin(theta) + Izz*cos(theta))*sin(theta), (Iyy*sin(theta) + Iyz*cos(theta))*sin(theta) + (Iyz*sin(theta) + Izz*cos(theta))*cos(theta)]])

In [20]:
def build_matrix(a11, a12, a13, a21, a22, a23, a31, a32, a33):
    matrix = np.array([
        [a11, a12, a13],
        [a21, a22, a23],
        [a31, a32, a33]
    ])
    return matrix

def build_matrix_symmetric(a11, a12, a13, a22, a23, a33):
    matrix_symmetric = np.array([
        [a11, a12, a13],
        [a12, a22, a23],
        [a13, a23, a33]
    ])
    return matrix_symmetric

In [21]:
def euler_to_rotation(alpha, beta, gamma):
    """
    Compute the rotation matrix from Euler angles.
    Here we use the Z-Y-X convention:
      R = Rz(alpha) @ Ry(beta) @ Rx(gamma)
    Angles are in radians.
    """
    Rz = np.array([
        [np.cos(alpha), -np.sin(alpha), 0],
        [np.sin(alpha),  np.cos(alpha), 0],
        [0,              0,             1]
    ])
    
    Ry = np.array([
        [ np.cos(beta), 0, np.sin(beta)],
        [ 0,            1, 0],
        [-np.sin(beta), 0, np.cos(beta)]
    ])
    
    Rx = np.array([
        [1, 0,              0],
        [0, np.cos(gamma), -np.sin(gamma)],
        [0, np.sin(gamma),  np.cos(gamma)]
    ])
    
    # Combined rotation matrix
    return Rz @ Ry @ Rx

def transform_inertia_tensor(I_body, m, R_disp, alpha, beta, gamma):
    """
    Transform the inertia tensor I_body (in the body frame) by:
      1. Rotating it using the Euler angles (alpha, beta, gamma).
      2. Applying the parallel axis theorem for a displacement R_disp.
    
    Parameters:
      I_body : 3x3 numpy array
          Inertia tensor in the body frame.
      m : float
          Mass of the body.
      R_disp : array-like of length 3
          Displacement vector (x, y, z) from the center of mass to the new axis.
      alpha, beta, gamma : float
          Euler angles in radians.
    
    Returns:
      I_transformed : 3x3 numpy array
          The transformed inertia tensor.
    """
    # Rotation matrix from Euler angles
    R = euler_to_rotation(alpha, beta, gamma)
    
    # Rotate the inertia tensor
    I_rot = R @ I_body @ R.T
    
    # Compute the parallel axis term
    d2 = np.dot(R_disp, R_disp)
    d_outer = np.outer(R_disp, R_disp)
    parallel_axis_term = m * (d2 * np.eye(3) - d_outer)
    
    # Combine rotated inertia with the parallel axis correction
    I_transformed = I_rot + parallel_axis_term
    return I_transformed

# Example usage
if __name__ == "__main__":
    # Define an example inertia tensor (in body frame)
    I_body = np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.5, 0.0],
        [0.0, 0.0, 2.0]
    ])
    
    mass = 1.0  # mass of the body
    
    # Define Euler angles in degrees and convert to radians
    alpha = np.deg2rad(30)
    beta  = np.deg2rad(45)
    gamma = np.deg2rad(60)
    
    # Define the displacement vector R = (x, y, z)
    R_disp = np.array([0.5, 0.2, 0.1])
    
    # Compute the transformed inertia tensor
    I_transformed = transform_inertia_tensor(I_body, mass, R_disp, alpha, beta, gamma)
    
    print("Transformed Inertia Tensor:")
    print(I_transformed)


Transformed Inertia Tensor:
[[ 1.63570752 -0.4201162   0.29717949]
 [-0.4201162   1.86179248  0.00366748]
 [ 0.29717949  0.00366748  1.6025    ]]
